<center><img src="./logo-cs.jpeg" width="400"  /></center>

<hr color="black">

<center><h1>Le module ETL de PistusResa</h1></center>

<hr color="black">

Les organisateurs des éditions précédentes de Pistus avaient conservé toutes les données d'inscription dans deux feuilles de calcul. 

Le module **ETL** est chargé d'**extraire (E)** les données des feuilles de calcul, de **transformer (T)** et de **charger (L, en anglais Load)** ces données dans la base de données de Pistus.

Les deux feuilles de calcul sont disponibles dans le dossier ``./data`` en tant que fichiers CSV.

Si vous regardez les fichiers, vous remarquerez immédiatement que :

* Certaines données sont répliquées. Le prénom, le nom et l'adresse électronique d'un étudiant se retrouvent dans les deux fichiers ;

* Certains étudiants (par exemple Ericka GUYOMARD) apparaissent plusieurs fois dans les deux fichiers parce qu'ils participent à plusieurs éditions de Pistus ou appartiennent à plus d'une association.

* Le sexe est exprimé de différentes manières : les hommes sont désignés par M, H, ou *garçon* et les femmes par F, W ou *fille*.


---

**Bon à savoir.** La redondance des données peut provoquer des **incohérences** dans une base de données. En effet, lorsque nous voulons modifier l'adresse électronique d'un étudiant et que cette adresse est présente dans plusieurs tables, nous pouvons oublier de mettre à jour certaines de ces occurrences.

---



Toutes ces anomalies dans les données sont appelées  **bruit** ; pratiquement tous les ensembles de données présentent une certaine quantité de bruit, qu'il faut traiter avant d'utiliser les données. En d'autres termes, nous devons **nettoyer les données** avant de les charger dans la base de données.

# La base de données Pistus

Dans le dernier TD, vous avez créé une base de données SQLite avec toutes les tables nécessaires. Si vous avez suivi les instructions à la lettre, vous devriez avoir sauvegardé la base de données dans un fichier nommé ``pistus.db``.

---

**Activité.** Copiez le fichier ``pistus.db`` dans le dossier ``./data``.

---

# Extraction de données

Votre objectif est de créer autant de Pandas DataFrames qu'il y a de tables dans le modèle physique de la base de données.  

---

**Question.** Écrivez un code pour : 

* lire les deux fichiers CSV dans deux DataFrames séparés :

* obtenir à partir de ces deux DataFrames les DataFrames correspondant aux tables de votre modèle physique.

👉 Dans les deux fichiers CSV, le délimiteur de colonne est un point-virgule (``;``).

---

In [46]:
# ÉCRIVEZ VOTRE CODE ICI
! pip install pandas

import pandas as pd

memberships_df = pd.read_csv("../data/student_memberships.csv", sep=';')
registrations_df = pd.read_csv("../data/student_registrations.csv", sep=';')

# Transformation des données 

Votre objectif est de nettoyer les données de chaque DataFrame.

## Doublons

Les DataFrames que vous avez obtenues à partir des fichiers CSV peuvent contenir des lignes en double.

---

**Question.** Écrivez un code pour supprimer les lignes en double de chaque DataFrame que vous avez obtenu dans la question précédente.

---

In [54]:
# ÉCRIVEZ VOTRE CODE ICI
memberships_df = memberships_df.drop_duplicates()
registrations_df = registrations_df.drop_duplicates()
memberships_df
registrations_df

,stud_number,first_name,last_name,gender,email,registration_date,payment_date,registration_fee,year
0,7719175,Eliane,CHOISNE,M,eliane.choisne@centralesupelec.fr,2015-10-08,2015-11-07,20.5,2016
1,7208231,Florian,COEFFARD,M,florian.coeffard@centralesupelec.fr,2015-10-01,2015-10-01,20.5,2016
2,5876984,Jacob,ROUSSIERE,M,jacob.roussiere@centralesupelec.fr,2015-10-10,2015-11-09,20.5,2016
3,2486399,Hortense,GUEFFIER,F,hortense.gueffier@centralesupelec.fr,2015-10-10,2015-10-10,20.5,2016
4,7874799,Timothée,COUILLAUD,M,timothee.couillaud@centralesupelec.fr,2015-09-27,2015-10-27,20.5,2016
...,...,...,...,...,...,...,...,...,...
895,8851393,Annie,ROULLIER,F,annie.roullier@centralesupelec.fr,2017-10-18,2017-10-18,23.0,2018
896,8446594,Alexia,CHAIGNEAU,F,alexia.chaigneau@centralesupelec.fr,2017-09-17,2017-09-17,23.0,2018
897,549362,Joanny,SUBILEAU,M,joanny.subileau@centralesupelec.fr,2017-09-24,2017-10-24,23.0,2018
898,8347453,Régine,GEORGET,M,regine.georget@centralesupelec.fr,2017-10-19,2017-10-19,23.0,2018



---

**Question.** Écrivez une liste d'assertions pour vérifier que vous avez :

* 602 étudiants.

* 11 associations.

* 602 adresses électroniques différentes.

* 3 éditions de Pistus.

* 900 inscriptions.

* 895 adhésions à des associations.

👉 Les assertions peuvent être écrites en utilisant l'instruction ``assert`` suivie d'une expression booléenne.

__Exemple.__

```python
assert 3 == 3
assert 5 > 1
```




---

In [48]:
# ÉCRIVEZ VOTRE CODE ICI
assert registrations_df.groupby('stud_number').count().shape[0] == 602
assert memberships_df.groupby('asso_name').count().shape[0] == 11
assert registrations_df.groupby('email').count().shape[0] == 602
assert registrations_df.groupby('year').count().shape[0] == 3
assert registrations_df.shape[0] == 900
assert memberships_df.shape[0] == 895

## Cohérence des informations relatives au sexe

Comme indiqué précédemment, les informations relatives au sexe ne sont pas cohérentes entre les lignes. 

* Les références ``M``, ``H`` ou ``garçon`` sont utilisées pour faire référence aux hommes.

* Les références ``F``, ``W`` ou ``fille`` sont utilisées pour faire référence aux filles.

Nous décidons que :

* les hommes seront toujours référencés comme ``M``.

* Les femmes seront toujours référencés comme ``F``.


---

**Question** : Écrivez le code pour rendre cohérentes les informations relatives au sexe.

---

In [55]:
# ÉCRIVEZ VOTRE CODE ICI
def normalize_gender(value):
    if pd.isna(value):
        return None
   
    if value in ["M", "H", "garçon"]:
        return "M"
    elif value in ["F", "W", "fille"]:
        return "F"
    else:
        return "Unknown"

registrations_df['gender'] = registrations_df["gender"].apply(normalize_gender)
memberships_df['gender'] = memberships_df["gender"].apply(normalize_gender)


---

**Question.** Écrivez des assertions pour vérifier que les informations relatives au sexe sont maintenant cohérentes.

---

In [56]:
# ÉCRIVEZ VOTRE CODE ICI
def compare_genders(genders):
    for gender in genders:
        if gender in ["H", "garçon", "W", "fille"]:
            return False
    return True

assert compare_genders(registrations_df['gender']) == True
assert compare_genders(memberships_df['gender']) == True

## Cohérence des dates

Les dates d'inscription et de paiement sont au format AAAA-MM-JJ. Nous voulons qu'elles soient exprimées sous la forme JJ-MM-AAAA.

---

**Question.** Écrivez le code pour changer le format des dates d'inscription et de paiement.

---

In [ ]:
# ÉCRIVEZ VOTRE CODE ICI



## Chargement des données dans la base de données

Nous sommes prêts à alimenter notre base de données Pistus avec des données propres.

---

**Question.** Écrivez le code pour importer les données dans la base de données Pistus. 

👉 Vérifiez que les données sont dans la base de données avec ``DB Browser for SQLite``.

👉 Rappelez-vous que les tables existent déjà dans votre base de données. Passez les arguments appropriés à la méthode ``to_sql``.

---

In [ ]:
# ÉCRIVEZ VOTRE CODE ICI



Bravo ! N'oubliez pas de fermer la connexion à la base de données et de passer à l'activité suivante.

In [ ]:
conn.close()